In [3]:
!pip -q install requests xarray netcdf4 dask rioxarray rasterio geopandas shapely fiona pyproj


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/s eta 0:00:00


In [36]:
!pip -q install requests xarray netcdf4 dask rioxarray rasterio geopandas shapely fiona pyproj

import os
from google.colab import files
uploaded = files.upload()   # choose iho.zip

# Get the uploaded filename dynamically (assumes only one file is uploaded)
uploaded_zip_file = list(uploaded.keys())[0]
base_name_without_ext = os.path.splitext(uploaded_zip_file)[0] # e.g., "Bay of Bengal (1)"

!rm -rf /content/iho_shp
!mkdir -p /content/iho_shp
!unzip -o "{uploaded_zip_file}" -d /content/iho_shp
!ls -l /content/iho_shp

import requests
import xarray as xr
import rioxarray
import geopandas as gpd

ERDDAP_GRIDDAP = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg.nc"

year = 2025  # ✅ CHANGE YEAR HERE

# Find the actual subdirectory created by the unzip command
# It typically corresponds to the name of the top-level folder inside the zip
extracted_contents = os.listdir('/content/iho_shp')
extracted_shp_folder = None
for item in extracted_contents:
    if os.path.isdir(os.path.join('/content/iho_shp', item)):
        extracted_shp_folder = item
        break

if extracted_shp_folder is None:
    raise FileNotFoundError("Could not determine the extracted shapefile folder.")

# Construct the full path to the .shp file
# Assuming the actual shapefile within the folder is named 'iho.shp'
shp_path = os.path.join('/content/iho_shp', extracted_shp_folder, 'iho.shp')
field = "name"
value = "Bay of Bengal"

out_folder = f"/content/{year}_monthly_sst"
os.makedirs(out_folder, exist_ok=True)

# Load polygon
gdf = gpd.read_file(shp_path).to_crs("EPSG:4326")
gdf = gdf[gdf[field] == value]
if gdf.empty:
    raise ValueError("Bay of Bengal polygon not found. Check attribute field/value.")

# Polygon bounds -> bbox (add small buffer)
west, south, east, north = gdf.total_bounds
buf = 0.25
west, east, south, north = west-buf, east+buf, south-buf, north-buf

# Build ERDDAP request URL (daily SST for one year, zlev=0)
start = f"{year}-01-01T12:00:00Z"
end   = f"{year}-11-30T12:00:00Z"

query = (
    f"?sst[({start}):1:({end})]"
    f"[(0):1:(0)]"
    f"[({south}):1:({north})]"
    f"[({west}):1:({east})]"
)
url = ERDDAP_GRIDDAP + query

daily_nc = os.path.join(out_folder, f"oisst_daily_{year}_bbox.nc")
print("Downloading:", url)

with requests.get(url, stream=True, timeout=900) as r:
    r.raise_for_status()
    with open(daily_nc, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)

print("Saved daily NetCDF:", daily_nc)

# Open and compute monthly mean
ds = xr.open_dataset(daily_nc, chunks={"time": 31})
sst = ds["sst"].isel(zlev=0)

monthly = sst.resample(time="MS").mean()

# Export each month as GeoTIFF clipped to polygon
for t in monthly.time.values:
    img = monthly.sel(time=t)
    img = img.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=False)
    img = img.rio.write_crs("EPSG:4326", inplace=False)

    img_clip = img.rio.clip(gdf.geometry, gdf.crs, drop=True)

    ym = str(xr.DataArray(t).dt.strftime("%Y%m").values)
    out_tif = os.path.join(out_folder, f"oisst_monthly_sst_{ym}_CLIPPED.tif")
    img_clip.rio.to_raster(out_tif, compress="DEFLATE")
    print("Saved:", out_tif)

print("DONE. GeoTIFF folder:", out_folder)

!zip -r "{year}_monthly_sst.zip" "{year}_monthly_sst"
from google.colab import files
files.download(f"{year}_monthly_sst.zip")

Saving Bay of Bengal.zip to Bay of Bengal (32).zip
Archive:  Bay of Bengal (32).zip
   creating: /content/iho_shp/Bay of Bengal/
  inflating: /content/iho_shp/Bay of Bengal/iho.cst  
  inflating: /content/iho_shp/Bay of Bengal/iho.dbf  
  inflating: /content/iho_shp/Bay of Bengal/iho.prj  
  inflating: /content/iho_shp/Bay of Bengal/iho.shp  
  inflating: /content/iho_shp/Bay of Bengal/iho.shx  
total 4
drwxrwxrwx 2 root root 4096 Jan 12 06:36 'Bay of Bengal'
Downloading: https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg.nc?sst[(2025-01-01T12:00:00Z):1:(2025-11-30T12:00:00Z)][(0):1:(0)][(5.48397101600807):1:(24.12766897644218)][(78.64822006250603):1:(95.29884011645345)]
Saved daily NetCDF: /content/2025_monthly_sst/oisst_daily_2025_bbox.nc
Saved: /content/2025_monthly_sst/oisst_monthly_sst_202501_CLIPPED.tif
Saved: /content/2025_monthly_sst/oisst_monthly_sst_202502_CLIPPED.tif
Saved: /content/2025_monthly_sst/oisst_monthly_sst_202503_CLIPPED.tif
Saved: /content/2025_monthl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install earthaccess geopandas shapely fiona pyproj xarray netcdf4 rioxarray rasterio

import os
from pathlib import Path
import earthaccess
import geopandas as gpd
import xarray as xr
import rioxarray  # enables .rio
from google.colab import files

# -------------------- YOU EDIT ONLY THESE --------------------
YEAR = 2025
FIELD = "name"
VALUE = "Bay of Bengal"
OUT_PARENT = "/content/MODIS_SST_MONTHLY"     # <- where to save outputs (local Colab path)
# -------------------------------------------------------------

# Official monthly dataset (PO.DAAC)
SHORT_NAME = "MODIS_AQUA_L3_SST_THERMAL_MONTHLY_4KM_DAYTIME_V2019.0"

def main():
    print("Please upload your shapefile (.zip) for the Bay of Bengal.")
    uploaded = files.upload()

    # Get the uploaded filename dynamically (assumes only one file is uploaded)
    uploaded_zip_file = list(uploaded.keys())[0]

    !rm -rf /content/iho_shp
    !mkdir -p /content/iho_shp
    !unzip -o "{uploaded_zip_file}" -d /content/iho_shp
    !ls -l /content/iho_shp

    # Find the actual subdirectory created by the unzip command
    extracted_contents = os.listdir('/content/iho_shp')
    extracted_shp_folder = None
    for item in extracted_contents:
        if os.path.isdir(os.path.join('/content/iho_shp', item)):
            extracted_shp_folder = item
            break

    if extracted_shp_folder is None:
        raise FileNotFoundError("Could not determine the extracted shapefile folder.")

    # Construct the full path to the .shp file (assuming 'iho.shp' inside the extracted folder)
    shp_path_colab = os.path.join('/content/iho_shp', extracted_shp_folder, 'iho.shp')

    # 1) Read polygon from shapefile
    gdf = gpd.read_file(shp_path_colab).to_crs("EPSG:4326")
    gdf = gdf[gdf[FIELD] == VALUE]
    if gdf.empty:
        raise ValueError(f'No feature found where {FIELD} == "{VALUE}". Check the shapefile attribute table.')

    # bbox helps search (MODIS monthly files are global, but bbox is still OK for filtering)
    west, south, east, north = gdf.total_bounds

    # 2) Login to Earthdata (will prompt if no credentials stored)
    earthaccess.login()

    # 3) Search monthly granules for the year
    granules = earthaccess.search_data(
        short_name=SHORT_NAME,
        temporal=(f"{YEAR}-01-01", f"{YEAR}-12-31"),
        bounding_box=(west, south, east, north),
    )
    print("Granules found:", len(granules))

    if len(granules) == 0:
        raise RuntimeError("No granules found. Try a different YEAR or remove bbox filter.")

    # 4) Download to local folder
    year_folder = Path(OUT_PARENT) / f"{YEAR}_monthly_sst"
    year_folder.mkdir(parents=True, exist_ok=True)

    files_downloaded = earthaccess.download(granules, local_path=str(year_folder), threads=8)
    print("Downloaded:", len(files_downloaded))
    print("Saved in:", year_folder)

    # 5) Convert each monthly NetCDF -> clipped GeoTIFF
    tif_folder = year_folder / "geotiff_clipped"
    tif_folder.mkdir(parents=True, exist_ok=True)

    for fp in files_downloaded:
        fp = str(fp)
        ds = xr.open_dataset(fp)

        if "sst" not in ds:
            raise ValueError(f"'sst' not found in {fp}. Variables: {list(ds.data_vars)}")

        sst = ds["sst"]

        # Detect coord names (common: lon/lat)
        x_dim = "lon" if "lon" in sst.dims else ("longitude" if "longitude" in sst.dims else None)
        y_dim = "lat" if "lat" in sst.dims else ("latitude" if "latitude" in sst.dims else None)
        if x_dim is None or y_dim is None:
            raise ValueError(f"Could not find lon/lat dims in {fp}. Dims are: {sst.dims}")

        sst = sst.rio.set_spatial_dims(x_dim=x_dim, y_dim=y_dim, inplace=False)
        sst = sst.rio.write_crs("EPSG:4326", inplace=False)

        # Exact polygon clip (no resampling; just mask/crop)
        sst_clip = sst.rio.clip(gdf.geometry, gdf.crs, drop=True)

        base = os.path.splitext(os.path.basename(fp))[0]
        out_tif = tif_folder / f"{base}_BoB.tif"

        sst_clip.rio.to_raster(str(out_tif), compress="DEFLATE")
        print("Saved:", out_tif)

    print("\nDONE. GeoTIFFs are here:\n", tif_folder)

    # Zip and download the results
    !zip -r "{YEAR}_monthly_sst_MODIS.zip" "{year_folder}"
    files.download(f"{YEAR}_monthly_sst_MODIS.zip")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2026.1.0 which is incompatible.
gcs

Saving Bay of Bengal.zip to Bay of Bengal.zip
Archive:  Bay of Bengal.zip
   creating: /content/iho_shp/Bay of Bengal/
  inflating: /content/iho_shp/Bay of Bengal/iho.cst  
  inflating: /content/iho_shp/Bay of Bengal/iho.dbf  
  inflating: /content/iho_shp/Bay of Bengal/iho.prj  
  inflating: /content/iho_shp/Bay of Bengal/iho.shp  
  inflating: /content/iho_shp/Bay of Bengal/iho.shx  
total 4
drwxrwxrwx 2 root root 4096 Jan 12 06:36 'Bay of Bengal'
Enter your Earthdata Login username: mdnuruzzaman.ges@gmail.com
Enter your Earthdata password: ··········
Granules found: 25


QUEUEING TASKS | :   0%|          | 0/25 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/25 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/25 [00:00<?, ?it/s]

Downloaded: 25
Saved in: /content/MODIS_SST_MONTHLY/2025_monthly_sst
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20241201_20241231.L3m.MO.SST.sst.4km.NRT_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20241201_20241231.L3m.MO.SST.sst.4km_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20250101_20250131.L3m.MO.SST.sst.4km.NRT_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20250101_20250131.L3m.MO.SST.sst.4km_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20250201_20250228.L3m.MO.SST.sst.4km.NRT_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20250201_20250228.L3m.MO.SST.sst.4km_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff_clipped/AQUA_MODIS.20250301_20250331.L3m.MO.SST.sst.4km_BoB.tif
Saved: /content/MODIS_SST_MONTHLY/2025_monthly_sst/geotiff

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>